In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd drive/MyDrive/

%cd Text_ming_project/

%ls

Mounted at /content/drive
/content/drive/MyDrive
/content/drive/MyDrive/Text_ming_project
amazon-2017.pdf  apple-2021.pdf                         microsoft-2017.pdf
amazon-2018.pdf  apple-2022.pdf                         microsoft-2018.pdf
amazon-2019.pdf  apple-2023.pdf                         microsoft-2019.pdf
amazon-2020.pdf  en_core_web_sm-3.8.0-py3-none-any.whl  microsoft-2020.pdf
amazon-2021.pdf  goog-10-k-2023.pdf                     microsoft-2021.pdf
amazon-2022.pdf  google-2017.pdf                        microsoft-2022.pdf
amazon-2023.pdf  google-2018.pdf                        microsoft-2023.pdf
apple-2017.pdf   google-2019.pdf                        requirements.txt
apple-2018.pdf   google-2020.pdf                        Text_mining_project.ipynb
apple-2019.pdf   google-2021.pdf
apple-2020.pdf   google-2022.pdf


In [ ]:
%pip install pdfplumber PyPDF2
%pip install spacy
%pip wheel https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.8.0/en_core_web_sm-3.8.0-py3-none-any.whl
%pip install bertopic
!python3 -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 71.7 MB/s eta 0:00:00
  File was already downloaded /content/drive/MyDrive/Text_ming_project/en_core_web_sm-3.8.0-py3-none-any.whl
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 33.3 MB/s eta 0:00:00
✔ Download and installat

In [ ]:
import pdfplumber
import re
from PyPDF2 import PdfReader, PdfWriter
import matplotlib.pyplot as plt
import spacy

In [ ]:
def page_extract(pdf_path, chunk_size):
  with pdfplumber.open(pdf_path) as pdf:

    toc_page = None #finding the table of content page

    for _, page in enumerate(pdf.pages):
        text = page.extract_text()
        if re.search(r'\btable of contents|index\b', text.lower(), re.IGNORECASE):
            toc_page = text
            break

  section = "Mine Safety Disclosures" #the last subheading that we want to get all the information from the 10Q form

  #print(toc_page)
  # To capture the page number right after the header
  part_i_page = re.search(rf"{section}.*?(\d+)", toc_page)

  count = int(part_i_page.group(1))

  reader = PdfReader(pdf_path)
  pdf_writer = PdfWriter()

  content = []

  for i in range(count + 2):
      a = pdf_writer.add_page(reader.pages[i]).extract_text()
      content.append(a)


  full_text = " ".join(content)

  words = full_text.split()  # Split the text into words
  chunks = []
  chunk = []

  current_size = 0

  for word in words:
        word_length = len(word) + 1
        if current_size + word_length > chunk_size:
            chunks.append(" ".join(chunk))
            chunk = []
            current_size = 0
        chunk.append(word)
        current_size += word_length


  if chunk:
        chunks.append(" ".join(chunk))

  return chunks


nlp = spacy.load('en_core_web_sm')

def pre_process(content):
    preprocess = []
    for i in range(len(content)):
      text = re.sub(r'\d+', '', content[i])
      text = re.sub(r'-', ' ', text)
      text = re.sub(r'[^\w\s]', '', text)
      text = re.sub(r'\s+', ' ', text).strip()
      text = re.sub(r'_+', '', text)
      text = text.lower()

      doc = nlp(text)

      #lemmatized_text = [token.lemma_ for token in doc]

      final = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]

      preprocess.append(" ".join(final))

    return preprocess


from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel

def train_lda_model(documents, num_topics, passes=1):
    dictionary = Dictionary(documents)

    corpus = [dictionary.doc2bow(doc) for doc in documents]
    dictionary[0]
    model = LdaModel(corpus=corpus, id2word=dictionary.id2token, num_topics=num_topics, passes=passes)
    return model

2017

In [ ]:
apple_2017 = page_extract("apple-2017.pdf",chunk_size=300)
apple_2017_process = pre_process(apple_2017)

lda_content_2017 = [content.split() for content in apple_2017_process]
model_2017 = train_lda_model(lda_content_2017,num_topics=10,passes=10)
model_2017.print_topics()

[(0,
  '0.033*"company" + 0.010*"new" + 0.010*"include" + 0.009*"property" + 0.009*"product" + 0.008*"party" + 0.008*"intellectual" + 0.007*"result" + 0.007*"reasonable" + 0.007*"stock"'),
 (1,
  '0.044*"company" + 0.021*"product" + 0.013*"information" + 0.010*"provide" + 0.009*"include" + 0.009*"failure" + 0.008*"component" + 0.008*"service" + 0.007*"able" + 0.007*"condition"'),
 (2,
  '0.025*"company" + 0.021*"law" + 0.018*"exchange" + 0.016*"file" + 0.015*"report" + 0.014*"act" + 0.013*"registrant" + 0.012*"pursuant" + 0.012*"mark" + 0.012*"check"'),
 (3,
  '0.059*"company" + 0.030*"product" + 0.012*"component" + 0.009*"partner" + 0.009*"risk" + 0.007*"service" + 0.007*"include" + 0.007*"store" + 0.007*"affect" + 0.006*"trade"'),
 (4,
  '0.031*"company" + 0.014*"financial" + 0.011*"service" + 0.010*"product" + 0.009*"filer" + 0.009*"increase" + 0.007*"base" + 0.007*"market" + 0.007*"operating" + 0.007*"computer"'),
 (5,
  '0.063*"company" + 0.021*"product" + 0.011*"market" + 0.011*"

In [ ]:
from bertopic import BERTopic

topic_model_2017 = BERTopic()
topics, probs = topic_model_2017.fit_transform(apple_2017_process)

topic_model_2017.get_topic_info().iloc[:, [0,1, 3]]

,Topic,Count,Representation
0,-1,108,"[company, product, market, service, result, co..."
1,0,48,"[apple, app, mac, io, store, tv, device, inclu..."
2,1,41,"[component, supply, company, partner, product,..."
3,2,24,"[information, company, confidential, loss, sec..."
4,3,21,"[registrant, file, report, exchange, act, purs..."
5,4,18,"[item, financial, factor, statement, look, for..."
6,5,17,"[intellectual, patent, property, right, compan..."
7,6,13,"[service, product, customer, software, company..."
8,7,13,"[law, regulation, international, jurisdiction,..."
9,8,12,"[foreign, currency, dollar, rate, denominate, ..."


2018

In [ ]:
apple_2018 = page_extract("apple-2018.pdf",chunk_size=300)
apple_2018_process = pre_process(apple_2018)

lda_content_2018 = [content.split() for content in apple_2018_process]
model_2018 = train_lda_model(lda_content_2018,num_topics=10,passes=10)
model_2018.print_topics()

[(0,
  '0.052*"company" + 0.024*"financial" + 0.024*"affect" + 0.023*"adversely" + 0.022*"condition" + 0.019*"result" + 0.016*"materially" + 0.013*"form" + 0.013*"product" + 0.011*"k"'),
 (1,
  '0.020*"company" + 0.018*"product" + 0.014*"law" + 0.011*"change" + 0.011*"future" + 0.010*"exchange" + 0.010*"patent" + 0.009*"new" + 0.009*"introduction" + 0.009*"net"'),
 (2,
  '0.055*"company" + 0.020*"product" + 0.017*"service" + 0.011*"include" + 0.010*"software" + 0.009*"time" + 0.008*"substantial" + 0.008*"result" + 0.008*"market" + 0.007*"foreign"'),
 (3,
  '0.043*"apple" + 0.026*"app" + 0.024*"tv" + 0.023*"mac" + 0.021*"device" + 0.018*"io" + 0.016*"company" + 0.014*"content" + 0.012*"store" + 0.012*"allow"'),
 (4,
  '0.045*"company" + 0.017*"component" + 0.015*"product" + 0.012*"item" + 0.010*"new" + 0.009*"content" + 0.008*"include" + 0.008*"source" + 0.007*"risk" + 0.007*"party"'),
 (5,
  '0.061*"company" + 0.039*"product" + 0.020*"service" + 0.014*"party" + 0.009*"software" + 0.009

In [ ]:
topic_model_2018 = BERTopic()
topics, probs = topic_model_2018.fit_transform(apple_2018_process)

topic_model_2018.get_topic_info().iloc[:, [0,1, 3]]

,Topic,Count,Representation
0,-1,92,"[company, product, foreign, sale, retail, incl..."
1,0,43,"[service, product, company, market, software, ..."
2,1,40,"[apple, app, store, mac, tv, io, device, pay, ..."
3,2,37,"[item, exchange, report, registrant, act, form..."
4,3,36,"[component, product, supply, partner, company,..."
5,4,18,"[patent, intellectual, property, right, compan..."
6,5,14,"[law, regulation, international, datum, requir..."
7,6,13,"[information, confidential, company, access, s..."
8,7,11,"[disaster, accident, natural, public, business..."
9,8,10,"[financial, condition, result, economic, credi..."


2019

In [ ]:
apple_2019 = page_extract("apple-2019.pdf",chunk_size=500)
apple_2019_process = pre_process(apple_2019)

lda_content_2019 = [content.split() for content in apple_2019_process]
model_2019 = train_lda_model(lda_content_2019,num_topics=10,passes=10)
model_2019.print_topics()

[(0,
  '0.043*"company" + 0.020*"service" + 0.017*"component" + 0.013*"increase" + 0.013*"price" + 0.012*"product" + 0.010*"file" + 0.009*"content" + 0.008*"include" + 0.008*"stock"'),
 (1,
  '0.041*"company" + 0.018*"content" + 0.016*"service" + 0.014*"offer" + 0.013*"apple" + 0.012*"store" + 0.012*"available" + 0.012*"iphone" + 0.012*"app" + 0.008*"digital"'),
 (2,
  '0.029*"company" + 0.018*"statement" + 0.012*"k" + 0.012*"form" + 0.011*"item" + 0.011*"look" + 0.011*"financial" + 0.009*"time" + 0.009*"factor" + 0.009*"risk"'),
 (3,
  '0.017*"company" + 0.012*"product" + 0.012*"service" + 0.012*"partner" + 0.007*"manufacturing" + 0.007*"significant" + 0.007*"cash" + 0.007*"primarily" + 0.007*"material" + 0.007*"price"'),
 (4,
  '0.051*"product" + 0.050*"company" + 0.015*"service" + 0.012*"new" + 0.010*"filer" + 0.009*"asset" + 0.009*"introduction" + 0.008*"inventory" + 0.008*"demand" + 0.008*"accelerate"'),
 (5,
  '0.049*"company" + 0.016*"product" + 0.009*"sale" + 0.009*"develop" + 

In [ ]:
topic_model_2019 = BERTopic()
topics, probs = topic_model_2019.fit_transform(apple_2019_process)

topic_model_2019.get_topic_info().iloc[:, [0,1, 3]]

,Topic,Count,Representation
0,-1,57,"[company, product, service, result, affect, fo..."
1,0,32,"[product, company, service, party, software, m..."
2,1,25,"[company, information, law, datum, loss, subje..."
3,2,19,"[item, report, statement, registrant, file, ac..."
4,3,12,"[component, company, manufacturing, supplier, ..."


2020

In [ ]:
apple_2020 = page_extract("apple-2020.pdf",chunk_size=500)
apple_2020_process = pre_process(apple_2020)

lda_content_2020 = [content.split() for content in apple_2020_process]
model_2020 = train_lda_model(lda_content_2020,num_topics=10,passes=10)
model_2020.print_topics()

[(0,
  '0.066*"company" + 0.020*"business" + 0.012*"product" + 0.011*"affect" + 0.010*"retail" + 0.009*"condition" + 0.009*"operation" + 0.009*"result" + 0.008*"financial" + 0.008*"adversely"'),
 (1,
  '0.039*"company" + 0.020*"stock" + 0.019*"product" + 0.018*"market" + 0.015*"price" + 0.013*"nasdaq" + 0.012*"llc" + 0.012*"note" + 0.011*"service" + 0.009*"time"'),
 (2,
  '0.024*"company" + 0.013*"product" + 0.012*"security" + 0.012*"section" + 0.012*"service" + 0.010*"act" + 0.010*"mark" + 0.010*"intellectual" + 0.010*"property" + 0.010*"d"'),
 (3,
  '0.039*"company" + 0.017*"result" + 0.017*"service" + 0.012*"statement" + 0.011*"forward" + 0.011*"k" + 0.011*"form" + 0.011*"include" + 0.010*"look" + 0.009*"risk"'),
 (4,
  '0.046*"company" + 0.027*"product" + 0.014*"sale" + 0.011*"condition" + 0.011*"service" + 0.010*"financial" + 0.010*"item" + 0.010*"affect" + 0.010*"adversely" + 0.008*"result"'),
 (5,
  '0.012*"result" + 0.012*"registrant" + 0.009*"financial" + 0.009*"condition" + 0

In [ ]:
topic_model_2020 = BERTopic()
topics, probs = topic_model_2020.fit_transform(apple_2020_process)

topic_model_2020.get_topic_info().iloc[:, [0,1, 3]]

,Topic,Count,Representation
0,-1,33,"[company, financial, foreign, result, content,..."
1,0,114,"[company, product, service, business, include,..."
2,1,19,"[item, report, registrant, act, stock, stateme..."


2021

In [ ]:
apple_2021 = page_extract("apple-2021.pdf",chunk_size=300)
apple_2021_process = pre_process(apple_2021)

lda_content_2021 = [content.split() for content in apple_2021_process]
model_2021 = train_lda_model(lda_content_2021,num_topics=10,passes=10)
model_2021.print_topics()

[(0,
  '0.049*"company" + 0.015*"business" + 0.015*"result" + 0.012*"operation" + 0.011*"impact" + 0.010*"risk" + 0.008*"condition" + 0.008*"accident" + 0.008*"release" + 0.007*"trade"'),
 (1,
  '0.038*"company" + 0.014*"registrant" + 0.010*"mark" + 0.009*"supply" + 0.009*"credit" + 0.008*"check" + 0.008*"affect" + 0.008*"non" + 0.008*"trade" + 0.007*"risk"'),
 (2,
  '0.028*"company" + 0.018*"iphone" + 0.010*"result" + 0.010*"use" + 0.008*"significant" + 0.008*"pro" + 0.007*"release" + 0.007*"component" + 0.007*"cash" + 0.007*"form"'),
 (3,
  '0.027*"company" + 0.027*"intellectual" + 0.025*"property" + 0.014*"right" + 0.009*"tax" + 0.009*"service" + 0.008*"believe" + 0.008*"obtain" + 0.007*"party" + 0.007*"product"'),
 (4,
  '0.033*"company" + 0.018*"datum" + 0.017*"information" + 0.010*"subject" + 0.009*"system" + 0.009*"use" + 0.009*"law" + 0.009*"customer" + 0.007*"security" + 0.007*"network"'),
 (5,
  '0.058*"company" + 0.025*"product" + 0.015*"market" + 0.011*"service" + 0.011*"ap

In [ ]:
topic_model_2021 = BERTopic()
topics, probs = topic_model_2021.fit_transform(apple_2021_process)

topic_model_2021.get_topic_info().iloc[:, [0,1, 3]]

,Topic,Count,Representation
0,-1,76,"[company, store, result, subject, change, cond..."
1,0,81,"[product, company, service, party, intellectua..."
2,1,30,"[item, registrant, stock, report, file, llc, n..."
3,2,26,"[component, manufacturing, supply, partner, co..."
4,3,21,"[pandemic, natural, covid, disaster, company, ..."
5,4,21,"[company, investigation, business, law, govern..."
6,5,13,"[employee, team, diversity, culture, commit, d..."
7,6,12,"[attack, access, security, information, unauth..."
8,7,11,"[financial, risk, credit, marketable, cash, fa..."
9,8,11,"[foreign, currency, dollar, tariff, rate, prod..."


2022

In [ ]:
apple_2022 = page_extract("apple-2022.pdf",chunk_size=300)
apple_2022_process = pre_process(apple_2022)

lda_content_2022 = [content.split() for content in apple_2022_process]
model_2022 = train_lda_model(lda_content_2022,num_topics=10,passes=10)
model_2022.print_topics()

[(0,
  '0.056*"company" + 0.041*"s" + 0.017*"business" + 0.014*"operation" + 0.013*"law" + 0.013*"product" + 0.011*"change" + 0.011*"regulation" + 0.010*"adversely" + 0.009*"affect"'),
 (1,
  '0.038*"company" + 0.017*"s" + 0.012*"ipad" + 0.012*"mac" + 0.011*"include" + 0.011*"system" + 0.010*"partner" + 0.010*"line" + 0.008*"pro" + 0.008*"primarily"'),
 (2,
  '0.046*"company" + 0.027*"s" + 0.020*"result" + 0.016*"financial" + 0.016*"operation" + 0.016*"condition" + 0.013*"business" + 0.010*"supplier" + 0.010*"materially" + 0.009*"adversely"'),
 (3,
  '0.073*"company" + 0.044*"s" + 0.033*"product" + 0.030*"service" + 0.011*"business" + 0.010*"sale" + 0.010*"include" + 0.009*"property" + 0.009*"customer" + 0.009*"party"'),
 (4,
  '0.040*"company" + 0.021*"market" + 0.017*"component" + 0.016*"product" + 0.015*"s" + 0.015*"stock" + 0.014*"result" + 0.012*"nasdaq" + 0.012*"llc" + 0.011*"note"'),
 (5,
  '0.035*"company" + 0.026*"s" + 0.020*"financial" + 0.020*"business" + 0.019*"condition" +

In [ ]:
topic_model_2022 = BERTopic()
topics, probs = topic_model_2022.fit_transform(apple_2022_process)

topic_model_2022.get_topic_info().iloc[:, [0,1, 3]]

,Topic,Count,Representation
0,-1,85,"[company, business, change, product, store, re..."
1,0,65,"[product, service, company, intellectual, prop..."
2,1,34,"[component, company, supply, product, partner,..."
3,2,31,"[adversely, condition, affect, financial, oper..."
4,3,25,"[registrant, item, report, stock, act, file, l..."
5,4,21,"[company, litigation, investigation, governmen..."
6,5,20,"[information, datum, confidential, security, a..."
7,6,16,"[accident, disaster, natural, industrial, inte..."
8,7,14,"[apple, iphone, ipad, base, service, line, mac..."


2023

In [ ]:
apple_2023 = page_extract("apple-2023.pdf",chunk_size=300)
apple_2023_process = pre_process(apple_2023)

lda_content_2023 = [content.split() for content in apple_2023_process]
model_2023 = train_lda_model(lda_content_2023,num_topics=10,passes=10)
model_2023.print_topics()

[(0,
  '0.069*"company" + 0.038*"product" + 0.036*"service" + 0.017*"include" + 0.011*"impact" + 0.010*"offer" + 0.010*"content" + 0.009*"sale" + 0.009*"cost" + 0.008*"result"'),
 (1,
  '0.026*"company" + 0.018*"security" + 0.017*"act" + 0.015*"report" + 0.014*"datum" + 0.013*"registrant" + 0.013*"market" + 0.012*"stock" + 0.012*"information" + 0.011*"section"'),
 (2,
  '0.040*"company" + 0.015*"content" + 0.015*"service" + 0.014*"party" + 0.013*"sale" + 0.011*"market" + 0.011*"iphone" + 0.011*"product" + 0.011*"digital" + 0.009*"application"'),
 (3,
  '0.031*"company" + 0.013*"term" + 0.013*"component" + 0.012*"subject" + 0.009*"reasonable" + 0.009*"commercially" + 0.009*"investigation" + 0.009*"government" + 0.008*"risk" + 0.008*"process"'),
 (4,
  '0.042*"company" + 0.015*"foreign" + 0.014*"business" + 0.013*"customer" + 0.012*"component" + 0.009*"certain" + 0.008*"issue" + 0.008*"supplier" + 0.008*"asset" + 0.008*"manage"'),
 (5,
  '0.082*"company" + 0.030*"product" + 0.025*"busine

In [ ]:
topic_model_2023 = BERTopic()
topics, probs = topic_model_2023.fit_transform(apple_2023_process)

topic_model_2023.get_topic_info().iloc[:, [0,1, 3]]

,Topic,Count,Representation
0,-1,109,"[company, product, sale, business, service, in..."
1,0,78,"[company, product, service, property, intellec..."
2,1,45,"[company, result, affect, business, financial,..."
3,2,36,"[item, registrant, statement, report, form, ac..."
4,3,29,"[component, company, product, supply, supplier..."
5,4,13,"[information, confidential, access, unauthoriz..."


trend across years

In [ ]:
import pandas as pd

data = {2017: apple_2017_process,2018:apple_2018_process, 2019: apple_2019_process, 2020: apple_2020_process, 2021: apple_2021_process, 2022: apple_2022_process, 2023: apple_2023_process}

def create_yearly_documents(data):
    documents = []

    for year, doc_list in data.items():
        for doc in doc_list:
            documents.append((doc, year))

    return documents

documents = create_yearly_documents(data)

# Convert the list of tuples into a DataFrame
df = pd.DataFrame(documents, columns=['text', 'year'])

print(df)

                                                   text  year
0     united statessecurities exchange commission wa...  2017
1     exchange act transition period commission file...  2017
2     cupertino california address principal executi...  2017
3     note note note note note note nasdaq stock mar...  2017
4     new york stock exchange llc new york stock exc...  2017
...                                                 ...   ...
1858  price volatility past continue future addition...  2023
1859  unrelated company operate performance price vo...  2023
1860  growth profitability company believe price sto...  2023
1861  companys board director company share repurcha...  2023
1862  companys stock decline significantly material ...  2023

[1863 rows x 2 columns]


In [ ]:
topic_model = BERTopic()

text = df['text'].tolist()

topics, probs = topic_model.fit_transform(text)

timestamps = df["year"].to_list()

topics_over_time = topic_model.topics_over_time(text, timestamps)

# Visualize topics over time
topic_model.visualize_topics_over_time(topics_over_time)

In [ ]:
topics_of_interest = [1,42, 48]

topic_model.visualize_topics_over_time(topics_over_time, topics = topics_of_interest)